# Obtención de los DataFrames de MovieLens

In [ ]:
# Importar todo lo necesario
import numpy as np
import pandas as pd
import scipy as sp
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## DataFrame de MovieLens

Se trata de obtener varios DataFrames del conjunto de datos de MovieLens que puedan ser compatibles con la librería de LightFM.

In [ ]:
# Obtención del dataframe de data
data_df = pd.read_csv('data/ml_data.csv', delim_whitespace=True, names=['Id Usuario','Id Película','Valoración','Fecha'])

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
#data_df

# Obtención del dataframe de ml_items
items_df = pd.read_csv('data/ml_items.csv', sep='|',
    names=['Id Película','Título','Fecha de estreno','Fecha DVD','iMDB','Género desconocido','Acción','Aventura','Animación','Infantil','Comedia', 'Crimen','Docuemntal','Drama','Fantasía','Cine negro','Horror','Musical','Misterio','Romance','Ciencia ficción','Thriller','Bélico','Western'],
    encoding='latin-1')

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
#items_df

# Junto los dos dataframes
df = pd.merge(data_df, items_df.drop_duplicates(['Id Película']), on="Id Película", how="left")

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
df.head()

## Obtención del modelo colaborativo

In [ ]:
# Obtención del dataframe train
train_df = pd.read_csv('data/train.csv', 
    delim_whitespace=True, names=['Id Usuario','Id Película','Valoración','Fecha']).drop('Fecha', axis=1)
#train_df

# Obtención del dataframe test
test_df = pd.read_csv('data/test.csv', delim_whitespace=True, names=['Id Usuario','Id Película','Valoración','Fecha'])
#test_df

In [ ]:
# Obtención del modelo colaborativo y de su precisión con LightFM
collab_model = LightFM(loss='warp')
collab_model.fit(sp.sparse.csr_matrix(train_df.values), epochs=30, num_threads=2)
test_precision = precision_at_k(collab_model, sp.sparse.csr_matrix(train_df.values), k=5).mean()
test_precision

In [ ]:
# Obtención de las predicciones del modelo colaborativo
scores = collab_model.predict(5, np.arange(3))
top_items = items_df['Título'][np.argsort(-scores)]
print(top_items)